<a href="https://colab.research.google.com/github/yyaroslavskiy/cuddly-chainsaw/blob/feature%2Fyan-drop-duplicates/eda/EDA_sber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA

In [ ]:
import numpy as np
import pandas as pd
import regex as re

## Работа над датасетом с врачами

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/yyaroslavskiy/cuddly-chainsaw/refs/heads/develop/parsing/sberparse/df.csv')

### Преобразование столбцов

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['experience'] = (
    df['experience'].astype(str)
      .str.extract(r'(\d+)')[0] # берём первую группу цифр
      .astype('Int64')
)

In [ ]:
df['price'] = (
    df['price'].astype(str)
      .str.replace(r'\s+', '', regex=True) # убираем пробелы-разделители в числе
      .str.replace(r'[^0-9]', '', regex=True) # оставляем только цифры
      .replace('', np.nan) # пустые -> NaN
      .astype('Int64')
)

In [ ]:
df.link = df.link.str.split('?').str[0]

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.speciality.value_counts()

In [ ]:
def split_speciality(val):
    if pd.isna(val): return []
    elif ',' in val: return [i.strip() for i in val.split(',')]
    else: return [i.strip() for i in re.split(r'(?<=[а-я])(?=[А-Я])', val)]

df.speciality = df.speciality.apply(split_speciality)

In [ ]:
df.speciality.value_counts().tail(20)

### Работа с пропусками

In [ ]:
df = df.dropna(subset = ['name'])

In [ ]:
df[df.speciality.isna()]

### Работа с дубликатами

In [ ]:
len(df[df.duplicated(subset=['link'], keep='first')])

In [ ]:
df = df.drop_duplicates(subset=['link'], keep='first')

In [ ]:
df

## Работа над датасетом с отзывами

### Преобразование столбцов

In [ ]:
df_reviews = pd.read_csv('https://raw.githubusercontent.com/yyaroslavskiy/cuddly-chainsaw/refs/heads/develop/parsing/sberparse/df_reviews.csv')

In [ ]:
df_reviews

In [ ]:
df_reviews.doctor_link = df_reviews.doctor_link.str.split('?').str[0]

In [ ]:
df_reviews.rate.value_counts()

In [ ]:
df_reviews.rate = df_reviews.rate.replace({
    'Плохо': 1,
    'Ниже среднего': 2,
    'Средне': 3,
    'Хорошо': 4,
    'Отлично': 5,
    'нет оценки': np.nan
})

In [ ]:
df_reviews.date = df_reviews.date.str.replace(r'00(\d{2})', r'20\1', regex=True)

In [ ]:
df_reviews.date = df_reviews.date.str.replace(r'02(\d{2})', r'20\1', regex=True)

In [ ]:
month_dict = {
    'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04',
    'мая': '05', 'июня': '06', 'июля': '07', 'августа': '08',
    'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'
}

df_reviews.date = pd.to_datetime(df_reviews.date.replace(month_dict, regex=True), format='%d %m %Y')

In [ ]:
df_reviews.info()

In [ ]:
df_reviews

### Работа с пропусками

In [ ]:
df_reviews[df_reviews.rate.isna() & df_reviews.comment.isna()]

In [ ]:
df_reviews = df_reviews.dropna(subset = ['rate', 'comment'], how = 'all')

In [ ]:
df_reviews[df_reviews.doctor_name.isna()]

In [ ]:
df_reviews

### Работа с дубликатами

In [ ]:
len(df_reviews[df_reviews.duplicated(subset=['doctor_link', 'comment'], keep='first')])

In [ ]:
df_reviews = df_reviews.drop_duplicates(subset=['doctor_link', 'comment'], keep='first')

In [ ]:
df_reviews

## Сохранение итоговых датасетов

In [ ]:
df.to_csv('sber_clean.csv')
df.to_csv('sber_reviews_clean.csv')